### Setting up a basic training loop - not using attention

In [ ]:
''' Changing directories '''
import os 
if 'BERT' not in os.getcwd():
    os.chdir('BERT')
print("Current working dir is {}".format(os.getcwd()))

In [ ]:
import pyaml
import onmt
import torch
from dataset import TextDataset
from encoder import Encoder 
from lib.huggingface.transformers import RobertaTokenizer, CamembertTokenizer
from torch.utils.data import DataLoader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with open(os.path.join(os.path.dirname(os.getcwd()), "config", "config.yml"), "r") as fd:
    config = pyaml.yaml.load(fd, Loader=pyaml.yaml.Loader)

In [ ]:
euro_parl_dataset = 'data/data-30k-default/' # specifying the dataset that we want to use

In [ ]:
text_dataset = TextDataset(euro_parl_dataset, is_train=True)

In [ ]:
english_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
french_tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

In [ ]:
def collate(data): 
    '''Collating function to be passed into the dataloader '''
    input_sentences, output_sentences = zip(*data)
    input_lengths = [len(sentence)+2 for sentence in input_sentences]
    output_lengths = [len(sentence)+2 for sentence in output_sentences]
    
    batch_size = len(input_sentences)
    
    max_input_lengths = max(input_lengths)
    max_output_lengths = max(output_lengths)
    
    input_idx_tensor = torch.zeros((batch_size, max_input_lengths), dtype=torch.long)
    output_idx_tensor = torch.zeros((batch_size, max_output_lengths), dtype=torch.long)
    
    for idx, (sentence_len, input_sentence) in enumerate(zip(input_lengths, input_sentences)): 
        input_idx_tensor[idx, :] = torch.tensor(english_tokenizer.encode(input_sentence) + [1]*(max_input_lengths-sentence_len))

    
    for idx, (sentence_len, output_sentence) in enumerate(zip(output_lengths, output_sentences)): 
        output_idx_tensor[idx, :] = torch.tensor(french_tokenizer.encode(output_sentence) + [1]*(max_output_lengths-sentence_len))

    return (input_idx_tensor, output_idx_tensor)

In [ ]:
dataloader = DataLoader(text_dataset, batch_size=64,
                        shuffle=True, num_workers=4, collate_fn=collate)

In [ ]:
try:
    del english_encoder
except:
    pass 

#### Specifying the encoding and decoding models

In [ ]:
english_encoder = Encoder("english")
french_encoder = Encoder("french")

In [ ]:
print(english_encoder._modules['model'].embeddings)

In [ ]:
embeddings_fr = onmt.modules.embeddings(self, word_vec_size,
                 word_vocab_size,
                 word_padding_idx,
                 position_encoding=True):

In [ ]:
french_decoder = onmt.decoder.TransformerDecoder(**config["small_transformer"])  
english_decoder = onmt.decoder.TransformerDecoder(**config["small_transformer"])  

In [ ]:
def loss_fn(encoder_output_english, encoder_output_french, decoder_output_french, decoder_output_english):
    '''Adversarial Loss'''
    return None 

In [ ]:
def train(encoder_model, decoder_model, data_iter): 
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for batch in data_iter:
        optimizer.zero_grad()
        english_sentences, french_sentences = batch
        
        # using the pooled output 
        _, encoder_output_english = encoder_model(english_sentences)
        #_, encoder_output_french = encoder_model(french_sentences)
        
        
        #loss = loss_fn(output, target)
        #loss.backward()
        #optimizer.step()

        return

In [ ]:
train(english_encoder, None, dataloader)